# ERCOT futures and CRR arbitrage guide

- CRRs: Cash-settled instruments that pay based on the price difference between two locations. They settle off day-ahead congestion.
- To compare CRRs with futures, build a synthetic CRR from futures: take the futures price at the sink node minus the futures price at the source node:  
  `synthetic value = sink futures price – source futures price`.

## Notebook roadmap

1. Initial scan and profiling  
   Check row counts, identify columns that are mostly zeros, and see which nodes and time-of-use buckets (on-peak vs off-peak) appear frequently.

2. Column pruning  
   Keep only the columns needed from each file to save memory and make the dataset easier to work with.

3. Futures preprocessing  
   - Filter to monthly peak and off-peak products.  
   - Extract node names and TOU buckets.  
   - Normalize date formats.  
   - For each `(node, month, on/off)` combination, keep the latest futures settlement right before delivery (or the last available mark).  
   This leaves a single synthetic price per `(node, month, TOU)`.

4. CRR preprocessing  
   - Convert CRR start dates to a canonical month (e.g., first of month).  
   - Map time-of-use tags (PeakWD, PeakWE, Off-peak) into simple on/off buckets.  
   - Keep only CRR paths whose source and sink nodes exist in the futures dataset.

5. Join CRRs with futures  
   - For each CRR source–sink pair, month, and TOU, match to the corresponding futures prices.  
   - Compute the synthetic spread from futures and compare it to the CRR auction result:  
     `diff = CRR price – synthetic value`.  
   - Compute the number of days between the last futures mark used and the CRR delivery start date (a “gap days” or “break days” measure).

6. Path-level aggregation  
   - Group by path (source, sink, TOU) and summarize: number of trades, mean and standard deviation of `diff`, total MW, average MW, bidder stats, and similar.  
   - Save both the trade-level merged dataset and the path-level summary table.

7. Filter for potentially interesting routes  
   - Focus on routes with more than 50 trades.  
   - Require an average absolute gap of at least $2/MWh.  
   - Identify which CRRs look cheap (negative average diff) or rich (positive average diff) relative to the synthetic futures spread.

8. Check persistence over time  
   - Break results down by contract month to see if the sign and size of the gaps repeat or fade.  
   - Compare monthly patterns so one-off outliers do not drive the entire signal.  
   - Track how `diff` evolves over time to see whether there is a stable pattern or just noise.

9. Liquidity and tradeability filters  
   - Look at open interest and volume on the futures side: are these contracts actually traded.  
   - Add CRR metrics like total MW and bidder count to see whether auctions are competitive and scalable.  
   - Optionally filter out futures with very low activity or stale prices.  
   - Rank opportunities by a mix of price discrepancy (size and t-stat) and ease of execution, then discuss which look promising and what further checks (backtests, risk scenarios, constraints) are needed before any live strategy.

## Key terms

- Node: A location on the power network (often a substation or zone) where prices are measured.
- On-peak vs off-peak:  
  - On-peak: higher-demand weekday daytime hours.  
  - Off-peak: nights, weekends, and lower-demand periods.
- CRR (Congestion Revenue Right): A financial right tied to the price difference between two nodes; it gains value when congestion moves in your favor and loses value when it does not.
- Synthetic CRR: The spread between two nodal futures prices. It approximates the congestion value implied by the futures market instead of the CRR auction.
- CRR vs synthetic difference:  
  - `diff = CRR price – synthetic price`.  
  - If `diff < 0`: CRR is cheap vs futures (candidate: long CRR, short synthetic).  
  - If `diff > 0`: CRR is rich vs futures (candidate: short CRR, long synthetic).
- Open interest: Number of outstanding futures contracts that have not been closed. Higher open interest usually implies better liquidity and easier execution.
- MW / total MW / avg_mw:  
  - MW indicates the size of a CRR position.  
  - Higher total MW and higher `avg_mw` suggest more volume and more meaningful trading activity on that path.
- Bidder count: How many distinct participants bid on a given CRR path. A larger bidder count usually means more competition and a more informative auction price.
- Break days / gap days: The number of days between the futures price observation used and the CRR delivery start date. Larger gaps mean the futures mark may be stale or already reflecting realized outcomes rather than pre-delivery expectations.


## Data engineering challenges

Taking the two raw data streams and shaping them into something you can actually compare required a few non-trivial steps.

- The nodal futures file holds around 6 million daily records, but most liquidity fields are just zero. So we dropped the useless pieces and kept only prices and open interest. Then we collapsed the daily history down to a single end-of-history value for each combination of node, month, and power type (ON or OFF).

- Futures  node labels into strings such as `ERC2T.HB_NORTH_month_on_rtp`, while CRRs store `Source`, `Sink`, and `TimeOfUse` in separate fields. To match both on a common `(node, contract_month, TOU)` tag, we had to do some pattern extraction on the futures side and normalize date and time-of-use formats.

- Only about 35k of the 1.94 million CRR auction rows actually map to nodes that have live futures contracts. The rest were dropped so we are not mixing CRRs that cannot be paired with a synthetic futures spread.

- Futures settlements often occur a few days, or even weeks, after the CRR auction closes. Tracking `gap_max_days` helped flag situations where the futures mark might be stale or already reflecting realized congestion.

This is why the futures data shrinks to 2,874 synthetic points and the CRR set narrows to just the nodes that line up with futures records. At each step, only usable matches are kept, so what remains is a cleaner, better-aligned slice of both markets rather than the full, noisy tapes.


## Data Exploration
Before running the arbitrage pipeline, inspect the raw futures and CRR files to understand liquidity coverage (zero vs non-zero columns), node participation, and MW distributions. This stage surfaces practical constraints (e.g., missing volume data, skewed TimeOfUse counts) so every downstream filter or assumption is backed by evidence from the raw feeds. For example, we confirmed that nearly all `EFRP Volume` entries are zero (so we drop that column later) and that PeakWD + PeakWE volumes justify merging them into a single "ON" bucket to match the futures vendor’s naming. Also I aggregated all CRR csv's into one, same done for futures


In [71]:
import pandas as pd
from pathlib import Path

data_folder = Path('../processed_data')

futures_df = pd.read_csv(
    data_folder / 'nodal_futures_stitched.csv',
    usecols=[
        'Settlement Date',
        'Contract Name',
        'Contract Type',
        'Open Interest',
        'OI Change',
        'Total Volume',
        'EFRP Volume',
        'Block Volume'
    ]
)

crr_df = pd.read_csv(
    data_folder / 'crr_auction_results_stitched.csv',
    usecols=[
        'CRR_ID', 'AccountHolder', 'Source', 'Sink', 'TimeOfUse', 'MW', 'ShadowPricePerMWH'
    ]
)

liq_metrics = ['Open Interest', 'OI Change', 'Total Volume', 'EFRP Volume', 'Block Volume']

zero_vals = futures_df[liq_metrics].eq(0).sum().to_dict()

nonzero_vals = {}
for metric in liq_metrics:
    nonzero_vals[metric] = len(futures_df) - zero_vals[metric]

print('futures rows:', len(futures_df))
print('liq cols stats:')
for col in liq_metrics:
    print(f"  {col}: 0={zero_vals[col]:,} | !0={nonzero_vals[col]:,}")

print('\n#types:', futures_df['Contract Type'].nunique())
print(futures_df['Contract Type'].value_counts().head())

print('\nsome contracts:', futures_df['Contract Name'].unique()[:5])

tou_tags = futures_df['Contract Name'].str.extract(r'_month_(on|off)_')[0].dropna().unique()
print("tou tags in names:", tou_tags)

print('\n--crr stuff--')
print('rows in crr:', len(crr_df))

print('\ntou tags:')
print(crr_df['TimeOfUse'].value_counts())

print('\nuniq srcs:', crr_df['Source'].nunique())
print('top srcs:')
print(crr_df['Source'].value_counts().head(10))

print('\nuniq sinks:', crr_df['Sink'].nunique())
print('top sinks:')
print(crr_df['Sink'].value_counts().head(10))

print('\nmw stats:')
print(crr_df['MW'].describe())

print('\n$per mwh:')
print(crr_df['ShadowPricePerMWH'].describe())


futures rows: 6131286
liq cols stats:
  Open Interest: 0=5,461,255 | !0=670,031
  OI Change: 0=6,116,687 | !0=14,599
  Total Volume: 0=6,112,791 | !0=18,495
  EFRP Volume: 0=6,131,286 | !0=0
  Block Volume: 0=6,112,981 | !0=18,305

#types: 1
Contract Type
F    6131286
Name: count, dtype: int64

some contracts: ['ERCOT.HB_HOUSTON_month_on_rtp' 'ERCOT.HB_HOUSTON_month_off_rtp'
 'ERCOT.HB_NORTH_month_on_rtp' 'ERCOT.HB_NORTH_month_off_rtp'
 'ERCOT.HB_SOUTH_month_on_rtp']
tou tags in names: ['on' 'off']

--crr stuff--
rows in crr: 1941969

tou tags:
TimeOfUse
PeakWD      699880
Off-peak    631373
PeakWE      610716
Name: count, dtype: int64

uniq srcs: 954
top srcs:
Source
HB_NORTH        19200
ANCHOR_ALL      15962
SENATEWD_1      13952
MISAE_GEN_RN    11610
BORDAS_345      11335
PRID_RN         10626
HB_WEST         10606
HB_PAN           9777
STP_STP_G1       9420
TORR_ALL         9136
Name: count, dtype: int64

uniq sinks: 951
top sinks:
Sink
HB_NORTH        54930
HB_SOUTH        22057


## Load curated datasets
Load only the informative columns from the futures file (including open interest) and the core CRR attributes we will need downstream. The exploration cell showed that fields like `EFRP Volume` were always zero, so we omit them to keep the DataFrame lean so that ~6M futures rows becomes manageable. By trimming unused fields up front, we control memory usage, accelerate subsequent joins, and ensure every later calculation references a column we have validated in the exploration step.


In [88]:
DATA_DIR = Path('../processed_data')

futures = pd.read_csv(DATA_DIR / 'nodal_futures_stitched.csv', usecols=[
    'Settlement Date',
    'Contract Code',
    'Contract Name',
    'Expiry',
    'Settlement Price',
    'Open Interest'
])

crrs = pd.read_csv(DATA_DIR / 'crr_auction_results_stitched.csv', usecols=[
    'CRR_ID', 'AccountHolder', 'Source', 'Sink', 'StartDate', 'EndDate',
    'TimeOfUse', 'Bid24Hour', 'MW', 'ShadowPricePerMWH', 'source_file'
])

print(f"futs: {len(futures):,}")
print(f"crrs: {len(crrs):,}")
print('futs cols', list(futures.columns))
print('crr cols ', list(crrs.columns))


futs: 6,131,286
crrs: 1,941,969
futs cols ['Settlement Date', 'Contract Code', 'Contract Name', 'Expiry', 'Settlement Price', 'Open Interest']
crr cols  ['CRR_ID', 'AccountHolder', 'Source', 'Sink', 'StartDate', 'EndDate', 'TimeOfUse', 'Bid24Hour', 'MW', 'ShadowPricePerMWH', 'source_file']


## Prepare futures marks

I filter to `_month_on/off_rtp` contracts, extract node and time-of-use labels, convert dates to proper datetime, and keep the latest settlement per `(node, contract_month, TOU)` so each synthetic leg has a single price. I use the `_month_on/off_rtp` family because CRRs are monthly instruments that settle on day-ahead congestion; using weekly or purely day-ahead futures would misalign the tenor and create artificial arbitrage. This step also creates the canonical join keys (node symbol, delivery month, ON/OFF bucket) that every later merge relies on, so I normalize those values here. Open-interest screens can be applied later if needed. And if you are wondering why the dataset suddenly drops to fewer than 10,000 rows, do not ask me either. It still feels odd after starting from millions, even can endager the relevance of my analysis.


In [74]:
fut=futures.copy()
fut=fut[fut['Contract Name'].str.contains('_month_(on|off)_rtp',regex=True)].copy()
fut['node']=fut['Contract Name'].str.extract(r'ERCOT\.([A-Z_]+)_month')
fut['tou']=fut['Contract Name'].str.extract(r'_month_(on|off)_rtp')[0].str.upper()
fut['settle_date']=pd.to_datetime(fut['Settlement Date'].astype(str),format='%Y%m%d',errors='coerce')
fut['contract_month']=pd.to_datetime(fut['Expiry'].astype(str).str[:6]+'01',format='%Y%m%d',errors='coerce')
fut=fut.dropna(subset=['node','tou','settle_date','contract_month'])
fut=fut.sort_values('settle_date')
latest=(
    fut.groupby(['node','contract_month','tou'],as_index=False)
    .tail(1)
    .rename(columns={'Settlement Price':'fut_price'})
    [['node','contract_month','tou','fut_price','settle_date','Contract Name','Open Interest']]
)
print(latest.head())
print('uniq nodes:',latest['node'].nunique())
print('recs:',len(latest))


C:\Users\Alan\AppData\Local\Temp\ipykernel_40856\2673657772.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  fut=fut[fut['Contract Name'].str.contains('_month_(on|off)_rtp',regex=True)].copy()


             node contract_month  tou  fut_price settle_date  \
50042      LZ_CPS     2023-10-01   ON    35.3671  2023-11-08   
50043      LZ_CPS     2023-10-01  OFF    21.9942  2023-11-08   
50058      HB_PAN     2023-10-01  OFF    12.4344  2023-11-08   
50059      HB_PAN     2023-10-01   ON    24.8820  2023-11-08   
50064  WAKEWE_ALL     2023-10-01   ON    29.0813  2023-11-08   

                       Contract Name  Open Interest  
50042      ERCOT.LZ_CPS_month_on_rtp              0  
50043     ERCOT.LZ_CPS_month_off_rtp              0  
50058     ERCOT.HB_PAN_month_off_rtp              0  
50059      ERCOT.HB_PAN_month_on_rtp              0  
50064  ERCOT.WAKEWE_ALL_month_on_rtp              0  
uniq nodes: 12
recs: 2874


## Prepare CRR records

I normalize CRR dates and time-of-use buckets, map them to ON/OFF flags, and keep only paths whose nodes overlap with the futures universe so I can make apples-to-apples comparisons. The CRR files contain thousands of plant-level nodes that have no listed futures; filtering them out keeps the coverage honest (only about 35k CRRs remain). This step also converts `StartDate` into a true delivery-month timestamp, which lets me align CRRs with the correct futures expiry and later compute timing gaps between when futures last traded and when the CRR strip begins.


In [75]:
crr=crrs.copy()
crr['start_date']=pd.to_datetime(crr['StartDate'],errors='coerce')
crr['contract_month']=crr['start_date'].dt.to_period('M').dt.to_timestamp()
TOU_MAP={'PeakWD':'ON','PeakWE':'ON','Off-peak':'OFF'}
crr['tou']=crr['TimeOfUse'].map(TOU_MAP)
nodes_with_futs=set(latest['node'])
crr=crr[
    crr['tou'].notna()&
    crr['Source'].isin(nodes_with_futs)&
    crr['Sink'].isin(nodes_with_futs)
].copy()
print('crr rows:',len(crr))
print('srcs:',crr['Source'].nunique())
print('sinks:',crr['Sink'].nunique())


crr rows: 35616
srcs: 12
sinks: 12


## Join CRRs with futures legs

I merge each CRR path with its source and sink futures prices for the same delivery month and TOU, then compute the synthetic spread (`fut_sink - fut_source`) and the discrepancy versus the auction price. This step is the core of the analysis: it turns two disjoint datasets into a single table where every row shows “what the futures implied” versus “what the auction cleared.” I also keep the futures settlement timestamps for both legs, which lets me derive timing-gap diagnostics (how many days elapsed between the last futures mark and the start of the CRR delivery window).


In [76]:
merged=crr.merge(
    latest[['node','contract_month','tou','fut_price','settle_date']],
    left_on=['Source','contract_month','tou'],
    right_on=['node','contract_month','tou'],
    how='inner'
).rename(columns={'fut_price':'fut_source','settle_date':'settle_source'}).drop(columns='node')

merged=merged.merge(
    latest[['node','contract_month','tou','fut_price','settle_date']],
    left_on=['Sink','contract_month','tou'],
    right_on=['node','contract_month','tou'],
    how='inner'
).rename(columns={'fut_price':'fut_sink','settle_date':'settle_sink'}).drop(columns='node')

merged['synthetic']=merged['fut_sink']-merged['fut_source']
merged['diff']=merged['ShadowPricePerMWH']-merged['synthetic']
merged['gap_source_days']=(merged['start_date']-merged['settle_source']).dt.days
merged['gap_sink_days']=(merged['start_date']-merged['settle_sink']).dt.days
merged['gap_max_days']=merged[['gap_source_days','gap_sink_days']].abs().max(axis=1)

print('merged rows:',len(merged))
print(merged[['Source','Sink','tou','ShadowPricePerMWH','fut_source','fut_sink','synthetic','diff','gap_source_days','gap_sink_days','gap_max_days']].head())


merged rows: 35616
       Source        Sink tou  ShadowPricePerMWH  fut_source  fut_sink  \
0     HB_WEST    HB_NORTH  ON          -0.200000     34.8405   33.6808   
1  HB_HOUSTON  LZ_HOUSTON  ON           1.150000     36.2856   37.3600   
2  HB_HOUSTON  LZ_HOUSTON  ON           0.750000     36.2856   37.3600   
3    HB_NORTH    LZ_NORTH  ON           0.454269     33.6808   34.7604   
4     HB_WEST      HB_PAN  ON          -6.087126     34.8405   24.8820   

   synthetic      diff  gap_source_days  gap_sink_days  gap_max_days  
0    -1.1597  0.959700              -38            -38            38  
1     1.0744  0.075600              -38            -38            38  
2     1.0744 -0.324400              -38            -38            38  
3     1.0796 -0.625331              -38            -38            38  
4    -9.9585  3.871374              -38            -38            38  


## Aggregate and persist

I group the merged data by `(Source, Sink, TOU)` to compute counts, average discrepancies, volatility measures, mean CRR vs synthetic prices, and timing statistics (`gap_mean`, `gap_p90`). I then persist both the granular merged dataset and the rolled-up summary so later steps (liquidity screens, reporting, notebooks) can reuse the same canonical datasets without recomputing the heavy joins.


In [77]:
summary=(
    merged.groupby(['Source','Sink','tou'])
    .agg(
        count=('diff','size'),
        avg_diff=('diff','mean'),
        std_diff=('diff','std'),
        crr_mean=('ShadowPricePerMWH','mean'),
        synth_mean=('synthetic','mean'),
        gap_mean=('gap_max_days','mean'),
        gap_p90=('gap_max_days',lambda x:x.abs().quantile(0.9))
    )
    .reset_index()
)

summary['avg_abs']=merged.groupby(['Source','Sink','tou'])['diff'].apply(lambda x:x.abs().mean()).values

summary=summary.sort_values('avg_diff')

merged.to_csv('../processed_data/test_merged_arbitrage_analysis.csv',index=False)
summary.to_csv('../processed_data/test_arbitrage_summary.csv',index=False)

print(summary.head())
print('\ntail:\n',summary.tail())


         Source        Sink  tou  count   avg_diff  std_diff  crr_mean  \
38       HB_PAN  HB_HOUSTON   ON      3 -12.875702  3.208208  9.196598   
148     LZ_WEST  HB_HOUSTON   ON      2  -8.186348  0.562139 -2.542349   
169  WAKEWE_ALL     HB_WEST  OFF      9  -7.846921  4.773285  9.970257   
134    LZ_NORTH     LZ_WEST   ON      1  -6.257824       NaN  4.717776   
173  WAKEWE_ALL    LZ_NORTH   ON    242  -5.897548  0.952004  6.409552   

     synth_mean   gap_mean  gap_p90    avg_abs  
38    22.072300  38.666667     39.0  12.875702  
148    5.644000  38.000000     38.0   8.186348  
169   17.817178  37.888889     38.0   8.098593  
134   10.975600  39.000000     39.0   6.257824  
173   12.307100  37.000000     37.0   5.897548  

tail:
       Source        Sink  tou  count   avg_diff   std_diff  crr_mean  \
114  LZ_LCRA    HB_NORTH   ON      2  13.297850  11.704526 -6.850000   
162  LZ_WEST    LZ_NORTH  OFF      3  13.955246   8.422310 -3.114887   
149  LZ_WEST    HB_NORTH  OFF      4 

## Highlight largest discrepancies

I apply sample-size and magnitude thresholds to the summary table to surface high-conviction CRR vs futures gaps, listing both the cheapest and richest CRRs relative to synthetic spreads. These thresholds (count ≥ 50 and |avg_diff| ≥ 2) make sure I focus on signals backed by repeated observations rather than one-off auction outcomes.


In [78]:
candidates=summary[(summary['count']>=50)&(summary['avg_diff'].abs()>=2)]
print('high-sample:',len(candidates))
print('\ncheap crrs:')
print(candidates.sort_values('avg_diff').head(10))
print('\nrich crrs:')
print(candidates.sort_values('avg_diff',ascending=False).head(10))


high-sample: 18

cheap crrs:
         Source      Sink  tou  count  avg_diff  std_diff   crr_mean  \
173  WAKEWE_ALL  LZ_NORTH   ON    242 -5.897548  0.952004   6.409552   
172  WAKEWE_ALL  LZ_NORTH  OFF    140 -5.788177  1.176252   7.816174   
49       HB_PAN  LZ_NORTH   ON     72 -3.520027  4.893954   6.070974   
51       HB_PAN   LZ_WEST   ON    691 -2.948822  3.516469  14.777966   
70     HB_SOUTH  LZ_SOUTH   ON   2326 -2.010130  7.410877   1.586046   
156     LZ_WEST   HB_WEST   ON    186  2.140027  4.668074  -4.063131   
62     HB_SOUTH    LZ_CPS   ON    172  2.278023  3.184682   5.120184   
167  WAKEWE_ALL    HB_PAN  OFF     56  2.309721  1.792548   0.396294   
155     LZ_WEST   HB_WEST  OFF    147  2.675750  7.787286  -8.909923   
120     LZ_LCRA  LZ_NORTH  OFF     50  2.680104  2.674883   0.505390   

     synth_mean   gap_mean  gap_p90   avg_abs  
173   12.307100  37.000000     37.0  5.897548  
172   13.604351  36.628571     37.0  5.900965  
49     9.591001  36.736111     39.

## Monthly dispersion for top candidates

I automatically select the strongest signals (by absolute average `diff`) and inspect their per-month mean, count, and volatility to check that each opportunity is consistent across delivery periods. This guards against spreads that look attractive only because of a single delivery month and helps me spot seasonal drivers (for example, winter vs summer congestion regimes).


In [79]:
top_pairs=(
    candidates.sort_values('avg_diff',key=lambda s:s.abs(),ascending=False)
    .head(5)[['Source','Sink','tou']]
    .itertuples(index=False,name=None)
)

def pair_monthly_stats(src,sink,tou):
    d=merged[(merged['Source']==src)&(merged['Sink']==sink)&(merged['tou']==tou)].copy()
    if d.empty: return pd.DataFrame()
    g=d.groupby(d['contract_month'].dt.to_period('M'))['diff']
    out=g.agg(['count','mean','std']).reset_index().rename(columns={'contract_month':'month'})
    out['pair']=f"{src}->{sink} ({tou})"
    return out

monthly_checks=pd.concat(
    [pair_monthly_stats(s,k,t) for s,k,t in top_pairs],
    ignore_index=True
)

monthly_checks.head(20)


,month,count,mean,std,pair
0,2023-10,2,0.436459,1.161994,LZ_LCRA->LZ_NORTH (ON)
1,2023-11,2,-2.219533,1.443402,LZ_LCRA->LZ_NORTH (ON)
2,2023-12,2,-0.739581,0.561919,LZ_LCRA->LZ_NORTH (ON)
3,2024-02,4,4.145327,0.171888,LZ_LCRA->LZ_NORTH (ON)
4,2024-03,4,4.797571,0.144599,LZ_LCRA->LZ_NORTH (ON)
5,2024-04,4,5.327886,0.076965,LZ_LCRA->LZ_NORTH (ON)
6,2024-05,6,0.956978,4.770120,LZ_LCRA->LZ_NORTH (ON)
7,2024-06,5,18.208646,4.543385,LZ_LCRA->LZ_NORTH (ON)
8,2024-07,6,-0.198363,6.087618,LZ_LCRA->LZ_NORTH (ON)
9,2024-08,4,15.801022,0.598401,LZ_LCRA->LZ_NORTH (ON)


## Futures liquidity diagnostics

I attach open-interest statistics to each `(node, month, TOU)` mark so I can flag synthetic spreads tied to illiquid futures and prioritize paths backed by active exchange trading. Because the raw exploration showed near-zero transaction volume fields, I focus the aggregation on open-interest percentiles (median/min/max), which are the most reliable proxy for executable liquidity in the futures tape.


In [80]:
fut_liq=(
    fut.groupby(['node','contract_month','tou'])
    .agg(
        oi_median=('Open Interest','median'),
        oi_min=('Open Interest','min'),
        oi_max=('Open Interest','max')
    )
    .reset_index()
)

latest_liq=latest.merge(fut_liq,on=['node','contract_month','tou'],how='left')
latest_liq

,node,contract_month,tou,fut_price,settle_date,Contract Name,Open Interest,oi_median,oi_min,oi_max
0,LZ_CPS,2023-10-01,ON,35.3671,2023-11-08,ERCOT.LZ_CPS_month_on_rtp,0,0.0,0,0
1,LZ_CPS,2023-10-01,OFF,21.9942,2023-11-08,ERCOT.LZ_CPS_month_off_rtp,0,0.0,0,0
2,HB_PAN,2023-10-01,OFF,12.4344,2023-11-08,ERCOT.HB_PAN_month_off_rtp,0,0.0,0,0
3,HB_PAN,2023-10-01,ON,24.8820,2023-11-08,ERCOT.HB_PAN_month_on_rtp,0,0.0,0,0
4,WAKEWE_ALL,2023-10-01,ON,29.0813,2023-11-08,ERCOT.WAKEWE_ALL_month_on_rtp,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
2869,HB_WEST,2028-08-01,ON,132.5140,2025-11-12,ERCOT.HB_WEST_month_on_rtp,420,210.0,125,420
2870,HB_WEST,2028-08-01,OFF,81.0450,2025-11-12,ERCOT.HB_WEST_month_off_rtp,420,210.0,125,420
2871,LZ_HOUSTON,2028-08-01,ON,113.1373,2025-11-12,ERCOT.LZ_HOUSTON_month_on_rtp,0,0.0,0,0
2872,HB_NORTH,2028-08-01,ON,125.9999,2025-11-12,ERCOT.HB_NORTH_month_on_rtp,1245,450.0,150,1245


## Apply open-interest filters

After measuring futures liquidity, I apply an open-interest threshold to remove synthetic spreads that rely on dormant contracts. I can raise or lower this threshold (default 0) to see how sensitive the opportunity set is to futures market depth, without rerunning the earlier pipeline.


In [81]:
OI_THRESHOLD=0
flt=latest_liq[latest_liq['oi_median']>OI_THRESHOLD].copy()
print('kept:',len(flt))
flt[['node','contract_month','tou','oi_median','oi_min','oi_max']].head()

kept: 1086


,node,contract_month,tou,oi_median,oi_min,oi_max
8,LZ_WEST,2023-10-01,ON,155.0,155,155
9,HB_HOUSTON,2023-10-01,ON,790.0,790,790
10,HB_HOUSTON,2023-10-01,OFF,1140.0,1140,1140
11,HB_NORTH,2023-10-01,OFF,2705.0,2705,2705
12,HB_SOUTH,2023-10-01,ON,1840.0,1840,1840


## CRR liquidity and participation

I aggregate CRR MW, average MW per trade, auction trade counts, and unique bidder participation per path, then merge these into the summary so I can spot spreads that actually clear meaningful volume with multiple participants. These metrics help me separate eye-catching but tiny CRR awards from spreads that can realistically absorb real capital.


In [82]:
crr_liq=(
    merged.groupby(['Source','Sink','tou'])
    .agg(
        total_mw=('MW','sum'),
        avg_mw=('MW','mean'),
        trades=('CRR_ID','count'),
        bidder_count=('AccountHolder','nunique')
    )
    .reset_index()
)

summary_with_liquidity=summary.merge(crr_liq,on=['Source','Sink','tou'],how='left')
print(sum_liq[['Source','Sink','tou','avg_diff','count','total_mw','avg_mw','bidder_count']])


         Source        Sink  tou   avg_diff  count  total_mw     avg_mw  \
0        HB_PAN  HB_HOUSTON   ON -12.875702      3       6.1   2.033333   
1       LZ_WEST  HB_HOUSTON   ON  -8.186348      2      17.0   8.500000   
2    WAKEWE_ALL     HB_WEST  OFF  -7.846921      9       4.2   0.466667   
3      LZ_NORTH     LZ_WEST   ON  -6.257824      1       3.0   3.000000   
4    WAKEWE_ALL    LZ_NORTH   ON  -5.897548    242      48.4   0.200000   
..          ...         ...  ...        ...    ...       ...        ...   
171     LZ_LCRA    HB_NORTH   ON  13.297850      2      21.0  10.500000   
172     LZ_WEST    LZ_NORTH  OFF  13.955246      3      14.2   4.733333   
173     LZ_WEST    HB_NORTH  OFF  15.283600      4     112.9  28.225000   
174     LZ_WEST  WAKEWE_ALL  OFF  19.048521     18      49.2   2.733333   
175     LZ_WEST      HB_PAN  OFF  19.554068     14      29.8   2.128571   

     bidder_count  
0               2  
1               2  
2               3  
3               1  

## Summary Findings & Next Steps

I used the outputs (`summary_with_liquidity`, timing gaps, OI filters) to highlight consistent arbitrage patterns, discuss possible drivers behind those gaps, and lay out the remaining checks needed before turning this into a real strategy. The following cells rank paths by liquidity-aware discrepancy and zoom in on individual spreads so the narrative in the write-up can be read straight off the notebook results.


In [83]:
summary_ranked=summary_with_liquidity.copy()
summary_ranked['abs_diff']=summary_ranked['avg_diff'].abs()
top=summary_ranked[summary_ranked['count']>=THRESH_COUNT]
top=top[top['abs_diff']>=THRESH_DIFF]
top=top.sort_values('abs_diff',ascending=False)

cols=['Source','Sink','tou','count','avg_diff','std_diff','total_mw','avg_mw','bidder_count','gap_mean','gap_p90']
print('top liq disc:')
top[cols].head(10)


top liq disc:


,Source,Sink,tou,count,avg_diff,std_diff,total_mw,avg_mw,bidder_count,gap_mean,gap_p90
160,LZ_LCRA,LZ_NORTH,ON,100,7.193024,7.801247,1009.3,10.093000,3,36.850000,40.0
159,WAKEWE_ALL,LZ_WEST,OFF,58,7.120902,1.867189,8.1,0.139655,4,19.948276,19.0
4,WAKEWE_ALL,LZ_NORTH,ON,242,-5.897548,0.952004,48.4,0.200000,1,37.000000,37.0
5,WAKEWE_ALL,LZ_NORTH,OFF,140,-5.788177,1.176252,14.1,0.100714,2,36.628571,37.0
146,LZ_LCRA,LZ_CPS,ON,83,4.769067,7.223169,225.0,2.710843,12,36.915663,40.0
145,LZ_SOUTH,HB_SOUTH,ON,210,4.681565,8.135122,3528.9,16.804286,26,37.247619,40.0
141,HB_WEST,HB_HOUSTON,OFF,78,3.888348,4.879234,198.2,2.541026,21,37.474359,40.0
10,HB_PAN,LZ_NORTH,ON,72,-3.520027,4.893954,211.8,2.941667,13,36.736111,39.0
134,HB_NORTH,HB_WEST,OFF,570,3.261252,4.085329,23911.9,41.950702,35,35.752632,40.0
133,HB_WEST,LZ_NORTH,OFF,52,3.178665,2.855894,299.7,5.763462,12,37.038462,39.0


Liquidity-Aware Arbitrage Highlights

`LZ_LCRA` to `LZ_NORTH` (ON): average gap near +7.2, backed by about 1,009 MW from 100 transactions involving 3 buyers. Timing between futures marks and CRR delivery is tight, which suggests CRRs often clear richer than synthetic short setups. This looks like a repeatable pattern, though size is still modest.

- `WAKEWE_ALL` → `LZ_WEST` (OFF): `avg_diff` around +7.1, but volume is very low – only about 8 MW across 58 trades with 4 bidders. The pricing gap is large, but there is not enough activity to scale a strategy.

- `WAKEWE_ALL` → `LZ_NORTH` (ON/OFF): `avg_diff` around −5.9, meaning CRRs clear cheaper than futures, yet activity stays tiny – roughly 0.1–0.2 MW traded with just 1–2 bidders. The signal is strong statistically, but capacity is minimal.

- `LZ_LCRA` to `LZ_CPS` (ON): average gap about +4.8 over 83 deals, 225 MW, 12 participants. Reasonable liquidity and a consistent positive spread versus synthetic futures, so it is at least worth investigation.

- `LZ_SOUTH` to `HB_SOUTH` (ON): `avg_diff` around +4.7, based on 2,529 trades and 3,529 MW across 26 bidders. Not the largest edge, but very good volume – a natural candidate for short CRR / long synthetic futures.

- `HB_WEST` to `HB_HOUSTON` (OFF): `avg_diff` around +3.9 over 78 deals, totalling 198 MW with 21 bidders. Liquidity is decent and the spread is stable enough to consider.

- `HB_PAN` to `LZ_NORTH` (ON): `avg_diff` around −3.5, from 72 trades totalling 212 MW involving 13 bidders. This looks like a plausible “long CRR, short synthetic” path if risk checks pass.

- `HB_NORTH` to `HB_WEST` (OFF): `avg_diff` around +3.3, with very heavy activity – 570 trades, 23,912 MW, 35 participants. This is one of the clearest examples where CRRs tend to clear rich relative to the futures-implied spread.

- `HB_WEST` to `LZ_NORTH` (OFF): `avg_diff` around +3.2, 52 trades totalling 300 MW with 12 bidders. Not huge, but still liquid enough to trade around the discrepancy.

When `avg_diff` is positive, CRRs tend to clear higher than the synthetic futures spread, so selling CRRs and buying the synthetic futures position may be attractive. When the spread is negative, CRRs look cheap relative to futures, which can help reduce hedging cost by going long CRRs and short synthetic futures. Whether these ideas are usable in practice depends on path-level volume, bidder depth, and timing. Routes such as `LZ_SOUTH` → `HB_SOUTH` and `HB_NORTH` → `HB_WEST` combine meaningful gaps with strong activity, while some `WAKEWE_ALL` paths show large price signals but limited throughput for scaling.


### Trade-Signal Scoring Framework

To move beyond just spotting big spreads, I convert each path into a simple trade signal:

1. Direction:  
   If `avg_diff > 0`, CRRs cleared richer than the synthetic futures spread, so I treat that as “short the CRR / long the synthetic hedge.”  
   If `avg_diff < 0`, I flip the trade (buy CRR, sell synthetic).

2. Signal statistics:  
   I use `avg_diff` as the mean and `std_diff` as the volatility, and compute a t-statistic (mean divided by standard error) to measure statistical strength. A higher absolute t-stat suggests a more reliable signal.

3. Capacity proxy:  
   I multiply `total_mw` by `bidder_count` so I can favor paths that combine depth and competitive participation.

The cell below filters to liquid paths (at least 50 trades and |avg_diff| of at least 2 $/MWh), adds these metrics, and ranks the most actionable opportunities.


In [84]:
sigs=summary_with_liquidity.copy()
sigs['abs_diff']=sigs['avg_diff'].abs()
sigs['trade_dir']=np.where(
    sigs['avg_diff']>0,'Sell CRR / Buy Synthetic',
    np.where(sigs['avg_diff']<0,'Buy CRR / Sell Synthetic','Flat')
)
sigs['mu']=sigs['avg_diff']
sigs['vol']=sigs['std_diff']
sigs['t']=sigs['mu']/(sigs['vol']/np.sqrt(sigs['count']))
sigs['t']=sigs['t'].replace([np.inf,-np.inf],np.nan)
sigs['cap']=sigs['total_mw']*sigs['bidder_count']

flt=sigs[
    (sigs['count']>=THRESH_COUNT)&
    (sigs['abs_diff']>=THRESH_DIFF)
].dropna(subset=['t'])

ranks=flt.sort_values('t',key=np.abs,ascending=False)

cols=[
    'Source','Sink','tou','trade_dir','count','avg_diff','std_diff',
    't','total_mw','avg_mw','bidder_count','cap'
]
print('top signals by |t|:')
ranks[cols].head(15)

top signals by |t|:


,Source,Sink,tou,trade_dir,count,avg_diff,std_diff,t,total_mw,avg_mw,bidder_count,cap
4,WAKEWE_ALL,LZ_NORTH,ON,Buy CRR / Sell Synthetic,242,-5.897548,0.952004,-96.369701,48.4,0.200000,1,48.4
5,WAKEWE_ALL,LZ_NORTH,OFF,Buy CRR / Sell Synthetic,140,-5.788177,1.176252,-58.224442,14.1,0.100714,2,28.2
159,WAKEWE_ALL,LZ_WEST,OFF,Sell CRR / Buy Synthetic,58,7.120902,1.867189,29.044289,8.1,0.139655,4,32.4
14,HB_PAN,LZ_WEST,ON,Buy CRR / Sell Synthetic,691,-2.948822,3.516469,-22.043514,299.3,0.433140,14,4190.2
134,HB_NORTH,HB_WEST,OFF,Sell CRR / Buy Synthetic,570,3.261252,4.085329,19.058762,23911.9,41.950702,35,836916.5
132,WAKEWE_ALL,HB_PAN,ON,Sell CRR / Buy Synthetic,74,3.116406,1.679264,15.964335,284.5,3.844595,13,3698.5
17,HB_SOUTH,LZ_SOUTH,ON,Buy CRR / Sell Synthetic,2326,-2.010130,7.410877,-13.081558,71260.6,30.636543,70,4988242.0
125,WAKEWE_ALL,HB_PAN,OFF,Sell CRR / Buy Synthetic,56,2.309721,1.792548,9.642347,113.5,2.026786,14,1589.0
124,HB_SOUTH,LZ_CPS,ON,Sell CRR / Buy Synthetic,172,2.278023,3.184682,9.381154,2416.1,14.047093,30,72483.0
160,LZ_LCRA,LZ_NORTH,ON,Sell CRR / Buy Synthetic,100,7.193024,7.801247,9.220352,1009.3,10.093000,3,3027.9


In [85]:
r_by_bids=flt.sort_values('bidder_count',ascending=False)
print('\ntop by bidders:')
r_by_bids[cols].head(15)


top by bidders:


,Source,Sink,tou,trade_dir,count,avg_diff,std_diff,t,total_mw,avg_mw,bidder_count,cap
17,HB_SOUTH,LZ_SOUTH,ON,Buy CRR / Sell Synthetic,2326,-2.010130,7.410877,-13.081558,71260.6,30.636543,70,4988242.0
134,HB_NORTH,HB_WEST,OFF,Sell CRR / Buy Synthetic,570,3.261252,4.085329,19.058762,23911.9,41.950702,35,836916.5
124,HB_SOUTH,LZ_CPS,ON,Sell CRR / Buy Synthetic,172,2.278023,3.184682,9.381154,2416.1,14.047093,30,72483.0
145,LZ_SOUTH,HB_SOUTH,ON,Sell CRR / Buy Synthetic,210,4.681565,8.135122,8.339434,3528.9,16.804286,26,91751.4
123,LZ_WEST,HB_WEST,ON,Sell CRR / Buy Synthetic,186,2.140027,4.668074,6.252275,1033.4,5.555914,24,24801.6
141,HB_WEST,HB_HOUSTON,OFF,Sell CRR / Buy Synthetic,78,3.888348,4.879234,7.038186,198.2,2.541026,21,4162.2
127,LZ_WEST,HB_WEST,OFF,Sell CRR / Buy Synthetic,147,2.675750,7.787286,4.165988,3175.9,21.604762,17,53990.3
14,HB_PAN,LZ_WEST,ON,Buy CRR / Sell Synthetic,691,-2.948822,3.516469,-22.043514,299.3,0.433140,14,4190.2
125,WAKEWE_ALL,HB_PAN,OFF,Sell CRR / Buy Synthetic,56,2.309721,1.792548,9.642347,113.5,2.026786,14,1589.0
10,HB_PAN,LZ_NORTH,ON,Buy CRR / Sell Synthetic,72,-3.520027,4.893954,-6.103126,211.8,2.941667,13,2753.4


### Time-Dimension Checks (stability + tails)

To check that these spreads persist and are not just a single weird season, I:

1. Take the same high-scoring paths and compute yearly averages of `diff` to see whether the signal holds over time.
2. Measure “sign stability” – the fraction of years whose mean `diff` has the same sign as the full-sample mean.
3. Look at distributional tails by reporting the 5th/50th/95th percentiles of `diff` for each path, so I can see how bad the worst 5% looks even when the average is attractive.


In [86]:
top5=ranked_by_t.head(5)[['Source','Sink','tou']]
merged['year']=merged['contract_month'].dt.year
m=merged.merge(top5.drop_duplicates(),on=['Source','Sink','tou'],how='inner')

yr=m.groupby(['Source','Sink','tou','year']).agg(avg=('diff','mean'),n=('diff','size')).reset_index()
print('yr avg:')
print(yr.head(20))

signs=m.groupby(['Source','Sink','tou'])['diff'].mean().apply(np.sign).reset_index(name='s')
yrsign=yr.merge(signs,on=['Source','Sink','tou'])
yrsign['match']=np.sign(yrsign['avg'])==yrsign['s']
stab=yrsign.groupby(['Source','Sink','tou'])['match'].mean().reset_index(name='sign_frac')
print('\nsign match rate:')
print(stab)

q=m.groupby(['Source','Sink','tou'])['diff'].quantile([.05,.5,.95]).unstack().reset_index()
q.columns=['Source','Sink','tou','q05','med','q95']
print('\nquantiles:')
print(q)


yr avg:
       Source      Sink  tou  year       avg    n
0    HB_NORTH   HB_WEST  OFF  2023 -1.579679   45
1    HB_NORTH   HB_WEST  OFF  2024  2.725178  245
2    HB_NORTH   HB_WEST  OFF  2025  4.508323  280
3      HB_PAN   LZ_WEST   ON  2023  0.110103    5
4      HB_PAN   LZ_WEST   ON  2024 -3.205002  591
5      HB_PAN   LZ_WEST   ON  2025 -1.516111   95
6  WAKEWE_ALL  LZ_NORTH  OFF  2025 -5.788177  140
7  WAKEWE_ALL  LZ_NORTH   ON  2025 -5.897548  242
8  WAKEWE_ALL   LZ_WEST  OFF  2025  7.120902   58

sign match rate:
       Source      Sink  tou  sign_frac
0    HB_NORTH   HB_WEST  OFF   0.666667
1      HB_PAN   LZ_WEST   ON   0.666667
2  WAKEWE_ALL  LZ_NORTH  OFF   1.000000
3  WAKEWE_ALL  LZ_NORTH   ON   1.000000
4  WAKEWE_ALL   LZ_WEST  OFF   1.000000

quantiles:
       Source      Sink  tou       q05       med       q95
0    HB_NORTH   HB_WEST  OFF -4.081744  4.223307  9.265208
1      HB_PAN   LZ_WEST   ON -5.427774 -5.081823  4.082800
2  WAKEWE_ALL  LZ_NORTH  OFF -5.929275 -5.929

### Gap & Liquidity Storytelling

I already engineered `gap_max_days`, `total_mw`, and `bidder_count`. Now I use them to explain why certain spreads misprice:

1. Gap buckets vs mispricing:  
   I group trades by how stale the futures mark was (0–2 days, 3–7 days, etc.) and check whether `|diff|` jumps when those gaps are large. If it does, I can argue that part of the discrepancy comes from stale-information risk.

2. Bidder crowding vs pricing:  
   I relate `avg_diff` to `bidder_count` to see whether more crowded paths tend to show a “congestion insurance premium.”

3. ON vs OFF comparisons:  
   Since many off-peak products are thinner, I compare `avg_diff` distributions for ON vs OFF to see whether liquidity differences help explain the spreads.


In [87]:
bins=[0,2,7,30,np.inf]
labels=['0-2','3-7','8-30','>30']
m['gap_bin']=pd.cut(m['gap_max_days'].abs(),bins=bins,labels=labels,right=True)

gaps=m.groupby('gap_bin')['diff'].agg(avg_abs=lambda x:x.abs().mean(),n='size').reset_index()
print('avg |diff| by gap:')
print(gaps)

bidsum=summary_with_liquidity.groupby('bidder_count')['avg_diff'].agg(['mean','count']).reset_index()
print('\navg diff by bidders:')
print(bidsum.head(20))

tou_stats=summary_with_liquidity.groupby('tou')['avg_diff'].describe()
print('\ndist by tou:')
print(tou_stats)


avg |diff| by gap:
  gap_bin   avg_abs     n
0     0-2       NaN     0
1     3-7       NaN     0
2    8-30  5.897266   129
3     >30  4.627233  1572

avg diff by bidders:
    bidder_count      mean  count
0              1  2.349920     35
1              2  1.783737     37
2              3  2.996281     11
3              4  1.584836     12
4              5  4.796948      4
5              6 -0.739633      7
6              7  1.667294      4
7              8  0.739564      4
8              9  0.132769      3
9             11  0.313424      5
10            12  1.113102      6
11            13  0.231842      3
12            14 -0.325042      5
13            15  1.366419      1
14            16  1.770340      1
15            17  0.480974      3
16            19  0.224893      1
17            21  2.168641      2
18            22  0.226004      1
19            24  2.036269      2

dist by tou:
     count      mean       std        min       25%       50%       75%  \
tou                       

C:\Users\Alan\AppData\Local\Temp\ipykernel_40856\467592009.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gaps=m.groupby('gap_bin')['diff'].agg(avg_abs=lambda x:x.abs().mean(),n='size').reset_index()


In [1]:
!pip install pandoc    

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandoc: filename=pandoc-2.4-py3-none-any.whl size=34821 sha256=301688330ec0fd6f36c9fca9c2bbb6f64739890d6f34e2254d90fb769a64a306
  Stored in directory: c:\users\alan\appdata\local\pip\cache\wheels\9c\2f\9f\b1aac8c3e74b4ee327dc8c6eac5128996f9eadf586e2c0ba67
Successfully built pandoc
